In [31]:
import numpy as np
import pandas as pd
import os

In [32]:
df = pd.read_csv('../data/hier.csv')

def getLen(dataframe, level):
    length = df[f'level_{level}'].max() + 1
    return(length)

df['level_1'] = df['level_1'] + getLen(df, 0)
df['level_2'] = df['level_2'] + getLen(df, 1)
df['level_3'] = df['level_3'] + getLen(df, 2)
df['level_4'] = df['level_4'] + getLen(df, 3)
df['level_5'] = df['level_5'] + getLen(df, 4)

df.head()

,gene,level_5,level_4,level_3,level_2,level_1,level_0
0,A1CF,397,129,26,14,1,0
1,AAAS,325,203,67,9,2,0
2,AAR2,542,126,43,14,1,0
3,AATF,598,149,43,14,1,0
4,ABAT,783,129,26,14,1,0


In [33]:
levels = [[],[],[],[],[],[]]
for i in range(0, 6):
    levels[i].extend(list(df[f'level_{i}'].unique()))
    levels[i].sort()

In [34]:
print('Size of each level')
for i, level in enumerate(levels):
    print(f'level {i}: {len(level)}')

Size of each level
level 0: 1
level 1: 2
level 2: 14
level 3: 53
level 4: 152
level 5: 657


In [35]:
genes = {}

for i, level in enumerate(levels):
    genes[f'level_{i}'] = {}
    for node in level:
        genes[f'level_{i}'][node] = list(df['gene'][df[f'level_{i}'] == node])

In [36]:
frames = []
for i in range(5):
    cols = df[[f'level_{i}', f'level_{i+1}']]
    cols = cols.drop_duplicates(subset=[f'level_{i}', f'level_{i+1}'], keep='first')
    cols.rename(columns={f'level_{i}': 'parent', f'level_{i+1}': 'child'}, inplace=True)

    frames.append(cols)

export = pd.concat(frames)

In [37]:
gene_dict = {
    **genes['level_0'],
    **genes['level_1'],
    **genes['level_2'],
    **genes['level_3'],
    **genes['level_4'],
    **genes['level_5'],
}
gene_ser = pd.Series(gene_dict)
gene_df = pd.DataFrame({'child':gene_ser.index, 'genes':gene_ser.values})

In [38]:
export['genes'] = ''
export['num_genes'] = 0

In [39]:
children = gene_dict.keys()
for child in children:
    export.loc[exp['child'] == child, 'num_genes'] = len(gene_dict[child])
    if child in gene_dict.keys():
        export.loc[exp['child'] == child, 'genes'] = ':'.join(gene_dict[child])

export.head()

,parent,child,genes,num_genes
0,0,1,A1CF:AAR2:AATF:ABAT:ABCA2:ABCB6:ABCB8:ABCB9:AB...,4252
1,0,2,AAAS:ACTA1:ACTA2:ACTB:ACTC1:ACTG1:ACTG2:ADRM1:...,716
0,1,14,A1CF:AAR2:AATF:ABAT:ABCA2:ABCB6:ABCB8:ABCB9:AB...,4252
1,2,9,AAAS:AMBRA1:ARPC1A:ARPC1B:ATG16L1:ATG16L2:BOP1...,86
25,2,13,ACTA1:ACTA2:ACTB:ACTC1:ACTG1:ACTG2:ANAPC11:AP1...,217


In [41]:
# Export list of genes to csv file
#exp.to_csv("cytoscape_input.csv", index=False, header=True)